__геокодинг с помощью данных OpenStreetMap Nominatim__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# sudo systemctl start postgresql
# sudo -u postgres nominatim serve  --project-dir /data/nominatim-planet 

In [2]:
# !pip install geopy

In [3]:
import re
# import json
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm

tqdm.pandas()
pd.options.display.max_colwidth = None 

## собираем список адресов для определения координат

In [4]:
# !ls data/location

In [5]:
# !ls data

In [6]:
ts = '2022-09-02_13-04'
data_path = f'data/avito_{ts}.xlsx'

### частичная обработка (только новые адреса)

In [7]:
# location_path = 'data/location/avito_2022-09-01_14-07_location.pkl' #  старый список адресов

# location = ( # список адресов из новых данных
#     pd.read_excel(data_path)[['adr']]
#     .drop_duplicates()
#     .merge(pd.read_pickle(location_path),on=['adr'],how='outer')
# )
# print(len(location))
# # location.sample(2)

# # список адресов без метки
# location_ = location[ location['coo'].isnull() ][['adr']].reset_index(drop=True)
# location_

### полный список адресов для обработки

In [8]:
location_ = pd.read_excel(data_path)[['adr']].drop_duplicates().reset_index(drop=True)
location_.sample(10)

,adr
1393,"проспект Генерала Острякова, 196"
852,"село Верхнесадовое, Севастопольская улица , 78"
233,"улица Колобова, 18/9"
1011,"улица Павла Корчагина, стр. 5"
909,"улица Лобанова, 25"
184,"улица Хрусталёва, 16"
1582,"Кача, улица Авиаторов, 34"
1572,"село Орловка, Качинское шоссе , 35/2"
274,"улица Степаняна, 11"
668,"улица Челюскинцев, 49"


## дополнительная очистка

In [9]:
location_['adr_'] = (
        location_['adr']
        .apply(lambda s: re.sub(r'\b[Сс]евастополь\b','',s))
        .apply(lambda s: re.sub(r'\bмикрорайон\b',' ',s))
        .apply(lambda s: re.sub(r'\bмуниципальный округ\b',' район ',s))
        .apply(lambda s: re.sub(r'\bкотт?еджный пос[ёе]лок\b',' ',s))
        .apply(lambda s: re.sub(r'\bпос[ёе]лок\b',' ',s))
        .apply(lambda s: re.sub(r'^','Севастополь, ',s))
        # .apply(lambda s: re.sub(r'[Сс]евастополь.*село\b','Крым, село ',s))
        .apply(lambda s: re.sub(r',\s*,',', ',s))
        .apply(lambda s: re.sub(r' +',' ',s))
    )

In [10]:
# location_[['adr_']].sort_values(by='adr_').to_csv('tmp/adr.csv',index=False)

In [11]:
# adr = 'Гагаринский муниципальный округ, микрорайон Казачья Бухта, Рубежная улица, 7'
# re.sub(r'.*муниципальный округ,','',adr)
# re.sub(r'микрорайон.*,','',adr)

## определяем координаты

In [14]:
# запускаем локальный геокодер

# FIXME: нужно настроить rewrite rule подменяющий /search?q=... на /search.php?q=... 

# sudo systemctl start postgresql
# sudo -u postgres nominatim serve  --project-dir /data/nominatim-planet 

# geolocator = Nominatim(domain='localhost:8088', scheme='http')

user_agent='Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
geolocator = Nominatim(user_agent=user_agent)


# adr = 'севастополь острякова 57'
# loc = geolocator.geocode(adr)
# (loc.latitude, loc.longitude)

In [15]:
def get_coo(adr, geolocator=geolocator):
    adr_ = adr.split(',')
    loc = geolocator.geocode(adr_) # пробуем определить координаты
    truncated = False
    while True:
        if not (loc is None): # нашел...
            return {'latitude':loc.latitude,'longitude':loc.longitude,'truncated':truncated }#,adr_
        
        # ...не нашел...
        if( len(adr_) < 3 ): break # адрес слишком короткий для сокращения
        adr_ = adr_[:-1] # выкидываем часть адреса (номер дома)
        loc = geolocator.geocode( adr_) # ... и пробуем ещё раз
        truncated = True
    
    return {'latitude':None,'longitude':None,'truncated':None, }

---

In [16]:
# df_ = location_['adr_'].sample(5).reset_index(drop=True)
# pd.concat([df_,df_.progress_apply(get_coo).apply(pd.Series)],axis=1)

In [18]:
# adr = '4-й Хомутов перeулок'
# adr = 'Античный бульвар, 24'
# adr = 'Английский бульвар, 22'
# adr = 'проспект Юрия Гагарина, 31'
# adr = 'село Вилино'
# adr = 'аллея Защитников 35-й Батареи, д. 11а'
# adr= 'СТ Ветеран, 4-я Ежевичная ул., 153'
# adr = 'Вилларис дель Мар'
# adr = 'Гагаринский муниципальный округ, микрорайон Казачья Бухта, Рубежная улица, 7'
# adr = 'Гагаринский район, Казачья Бухта, Рубежная улица, 7'
# adr = 'Рубежная улица, 7'
#adr = 'Любимовка, Южногородская улица, 36 к7'
# adr = 'бульвар Ататюрк, 275'
# adr = 'Эдельвейсовый проезд , 15'
# adr = 'ул. Генерала Лебедя,3'

# adr = 'Севастополь,'+adr
# get_coo(adr)

---

In [19]:
%%time

# ищем метку по адресу
location_ = pd.concat([location_, location_['adr_'].progress_apply(get_coo).apply(pd.Series)],axis=1)
location_.sample(4)

  0%|          | 0/1584 [00:00<?, ?it/s]

CPU times: user 13.3 s, sys: 656 ms, total: 14 s
Wall time: 18min 4s


,adr,adr_,latitude,longitude,truncated
207,"улица Астана Кесаева, 14В","Севастополь, улица Астана Кесаева, 14В",44.586769,33.453819,True
391,"улица Лизы Чайкиной, 97","Севастополь, улица Лизы Чайкиной, 97",44.594732,33.482095,True
665,"улица Вакуленчука, 12","Севастополь, улица Вакуленчука, 12",44.595739,33.487886,False
702,"улица Генерала Жидилова, 11","Севастополь, улица Генерала Жидилова, 11",44.604538,33.574684,False


In [20]:
# список адресов для которых не удалось найти метку 
location_[ location_['latitude'].isnull() ]

,adr,adr_,latitude,longitude,truncated
3,"аллея Защитников 35-й Батареи, д. 11а","Севастополь, аллея Защитников 35-й Батареи, д. 11а",NaN,NaN,None
86,"Севастопольская зона ЮБК, 27В","Севастополь, Севастопольская зона ЮБК, 27В",NaN,NaN,None
226,"улица Александра Шостака, 9","Севастополь, улица Александра Шостака, 9",NaN,NaN,None
273,"СТ Чайка-1, 248","Севастополь, СТ Чайка-1, 248",NaN,NaN,None
310,"коттеджный посёлок Вилларис дель Мар, 77","Севастополь, Вилларис дель Мар, 77",NaN,NaN,None
377,"улица Александра Шостака, 1","Севастополь, улица Александра Шостака, 1",NaN,NaN,None
401,"улица Александра Шостака, 43","Севастополь, улица Александра Шостака, 43",NaN,NaN,None
416,"село Вилино, улица Чапаева, 24","Севастополь, село Вилино, улица Чапаева, 24",NaN,NaN,None
506,"село Вилино, улица Чапаева, 1","Севастополь, село Вилино, улица Чапаева, 1",NaN,NaN,None
525,"улица Александра Шостака, 41","Севастополь, улица Александра Шостака, 41",NaN,NaN,None


---

In [21]:
# адреса, которые были обрезаны
# location_.dropna()[ location_.dropna()['truncated'] ]

In [22]:
# # собираем локации в один список
# location_ = pd.concat([
#     location[ ~location['coo'].isnull() ],
#     location_[ ~location_['coo'].isnull() ]
# ]).drop_duplicates().reset_index(drop=True)

In [23]:
col = ['adr','latitude','longitude','truncated']

# location.to_excel(f'data/avito_{ts}_location.xlsx', index=False)
location_.dropna().reset_index(drop=True)[col].to_pickle(f'data/location/avito_{ts}_location.pkl')

In [24]:
# location_.dropna().reset_index(drop=True)[col]

,adr,latitude,longitude,truncated
0,"улица Генерала Крейзера, д. 8, корп. 5",44.604490,33.512078,True
1,"проспект Победы, 29В",44.584041,33.563608,False
2,"улица Военных Строителей, стр. 6.3",44.567946,33.405590,True
3,"улица Генерала Мельника, 11А/1",44.576784,33.565291,True
4,"шоссе Лабораторное, д. 33, секц. 6",44.574609,33.580993,True
...,...,...,...,...
1541,"проспект Античный, д. 26, корп. 5",44.594117,33.435975,True
1542,"1-я Бастионная улица , 7",44.612911,33.550451,False
1543,"улица Павла Корчагина, 5",44.583350,33.433684,True
1544,"СТ Рубин, 58",44.540775,33.584603,False


---

In [ ]:
# from geopandas.tools import geocode
# geocode(adr, provider="nominatim" , user_agent = 'my_request')
